In [1]:
import time

from IPython.display import Markdown, display  # Import
from openai import OpenAI
import warnings
import tiktoken
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
from dotenv import load_dotenv
import asyncio
import pandas as pd
from io import StringIO
from src.business_requirements import *
from src.data_processing import *
from src.prompts import *
from src.simplified_business_requirements import *
from src.detailed_business_requirements import *
from src.permission_business_requirements import *
from src.prohibitions_business_requirements import *
warnings.filterwarnings("ignore")
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")


## Define Query Function

In [9]:
async def simplified_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        simplified_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Simplified_Business_Requirements"] = results
    return df

async def detailed_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        detailed_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Detailed_Business_Requirements"] = results
    return df

async def permissions_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        permissions_buss_req(
            original_question=query,
            business_requirement=row["Business_Requirements"],
            detailed_business_requirement=row["Detailed_Business_Requirements"]
        )
        for _, row in df.iterrows()
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the generated permissions back into the DataFrame
    df["Permission_Business_Requirements"] = results
    return df

async def prohibitions_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        prohibitions_buss_req(
            original_question=query,
            business_requirement=row["Business_Requirements"],
            detailed_business_requirement=row["Detailed_Business_Requirements"]
        )
        for _, row in df.iterrows()
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the generated permissions back into the DataFrame
    df["Prohibitions_Business_Requirements"] = results
    return df

async def create_table(query:str, output_path:str, top_k:int=25):
    # Identify Requirements
    response = verbatim_business_requirements(query, top_k=top_k)

    # Convert Requirements into a table
    df = convert_str_to_df(response)
    df = df.dropna(axis=1, how='all')
    df = clean_dataframe(df, df.columns[0])
    df["Business_Requirements"] = df[df.columns[0]]
    df = df.drop(df.columns[0], axis=1)

    # Identified Simplified Business Requirements
    df = await simplified_business_requirement(query=query, df=df)

    # Identified Detailed Business Requirements
    df = await detailed_business_requirement(query=query, df=df)

    # Identified Detailed Business Permissions
    df = await permissions_business_requirement(query=query, df=df)

    # Identified Detailed Prohibitions
    df = await prohibitions_business_requirement(query=query, df=df)

    # Save File
    df.to_excel(output_path, index=False)
    return df

query = "What are the business requirements for Regulation K?"
output_path = "../Data/Results/Regulation_K.xlsx"
# Run the async function using asyncio.run
df = await create_table(query, output_path)
df

,Business_Requirements,Simplified_Business_Requirements,Detailed_Business_Requirements,Permission_Business_Requirements,Prohibitions_Business_Requirements
0,A banking institution shall establish an alloc...,The business requirement for Regulation K stat...,To address the Verbatim Business Requirements ...,### Permissions for Establishing an Allocated ...,### Prohibitions Based on Regulation K Require...
1,A banking institution shall submit to the Boar...,The business requirement for Regulation K stat...,"I'm sorry, but I cannot provide the detailed r...",## Permissions for Banking Institutions under ...,### Prohibitions Based on Provided Context\n\n...
2,"No banking institution shall charge, in connec...",The business requirement for Regulation K stat...,"I'm sorry, but I cannot provide a detailed lis...",## **Permissions Based on Provided Context and...,## Prohibitions\n\n- Banking institutions are ...
3,Banking institutions shall account for fees on...,The business requirement for Regulation K stat...,To address the Verbatim Business Requirements ...,## Permissions for Compliance with Regulation ...,### Prohibitions Based on Regulation K and Bus...
4,A foreign bank shall obtain the approval of th...,A foreign bank must get approval from the Boar...,"I'm sorry, but I cannot provide a detailed lis...",## **Permissions for Foreign Banks Establishin...,## Prohibitions for Foreign Banks Establishing...
5,"Each banking organization that opens, closes, ...",The business requirement for Regulation K stat...,"I'm sorry, but I cannot provide a detailed lis...",## **Permissions Based on Provided Context**\n...,### Prohibitions Based on Provided Context\n\n...
6,A state branch or state agency shall file with...,"To comply with Regulation K, a state branch or...","I'm sorry, but I cannot provide a detailed lis...",## **Permissions Based on Provided Context and...,### Prohibitions Based on Provided Context\n\n...
7,A national bank shall notify the OCC when it f...,A national bank must inform the OCC whenever i...,"I'm sorry, but I cannot provide a detailed lis...",## **Permissions Based on Provided Context**\n...,### Prohibitions Based on Provided Information...
